In [ ]:
# !pip install torch

In [97]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

Here we instantiate an LSTM cell. W is the weight matrix that applies to the current input, U applies to the hidden state coming from the previous time period and b is the bias matrix. 

We then initialize the parameters. Biases can be initialized to zero but not the weight matrixes. Otherwise the derivative would be equal to zero and we could not update the weights. Here, we use the Xavier initialization which takes into account the size of the input and output to adjust the standard deviation of the normal distribution used to initialize the values. The mean of this distribution is, of course zero.

In [111]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Input gate parameters
        self.W_i = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_i = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i = nn.Parameter(torch.Tensor(hidden_size))

        # Forget gate parameters
        self.W_f = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_f = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))

        # Cell gate parameters
        self.W_c = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_c = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_c = nn.Parameter(torch.Tensor(hidden_size))

        # Output gate parameters
        self.W_o = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_o = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o = nn.Parameter(torch.Tensor(hidden_size))

        # Initialize weights
        self.init_weights()
    
    def init_weights(self):
        for name, param in self.named_parameters():
            if "W_" in name or "U_" in name:  
                nn.init.xavier_uniform_(param)
            elif "b_" in name:  
                nn.init.zeros_(param)
                
    def forward(self, x, hidden):
        h_prev, c_prev = hidden
        outputs = []

        for t in range(x.size(1)):
            x_t = x[:, t, :]

            # Input gate
            i_t = torch.sigmoid(x_t @ self.W_i + h_prev @ self.U_i + self.b_i)

            # Forget gate
            f_t = torch.sigmoid(x_t @ self.W_f + h_prev @ self.U_f + self.b_f)

            # Cell gate
            g_t = torch.tanh(x_t @ self.W_c + h_prev @ self.U_c + self.b_c)

            # Update cell state
            c_t = f_t * c_prev + i_t * g_t

            # Output gate
            o_t = torch.sigmoid(x_t @ self.W_o + h_prev @ self.U_o + self.b_o)

            # Update hidden state
            h_t = o_t * torch.tanh(c_t)

            outputs.append(h_t.unsqueeze(1))

            # Update states for next time step
            h_prev, c_prev = h_t, c_t

        outputs = torch.cat(outputs, dim=1)
        return outputs, (h_t, c_t)

Here, we create the LSTM layers. The for loop in the initialization stacks the cells to create the layers. In the end, we add a linear layer to convert the last hidden state into the output size we desire. 

In [158]:
class StackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, output_size=1):
        super(StackedLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.layers = nn.ModuleList()
        for i in range(num_layers):
            in_size = input_size if i == 0 else hidden_size
            self.layers.append(LSTM(in_size, hidden_size))

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden_states):
        for i, layer in enumerate(self.layers):
            h_prev, c_prev = hidden_states[i]
            x, (h, c) = layer(x, (h_prev, c_prev))  # Pass through each LSTM layer

        # Use the last time step's hidden state and apply the fully connected layer
        outputs = self.fc(x[:, -1, :])  # Shape: (batch_size, output_size)
        return outputs

In [116]:
data_merged = pd.read_csv('data_merged.csv')

/tmp/ipykernel_897/3479382016.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_merged = pd.read_csv('data_merged.csv')


For the training, we only keep the client present during the three years (400 000 people)

In [117]:
non_nan_data = data_merged.dropna()

In [121]:
subset_data = non_nan_data.iloc[:1000]

In [172]:
zero_count = (subset_data['fra_1123'] == 0).sum()
total_count = len(subset_data['fra_0123'])
percentage_of_zeros = (zero_count / total_count) * 100

print(f"Percentage of zeros: {percentage_of_zeros}%")

Percentage of zeros: 87.8%


We now need to reshape the data so that it can be fed into the network

In [160]:
# Extract time-dependent columns (these will be the targets)
time_columns = [col for col in subset_data.columns if col.startswith('fra_')]

# Extract static columns (these will be the features)
static_columns = ['Age22', 'category_Enfant', 'category_Homme', 'category_Femme']  # Static features

# Create the static feature matrix
static_data = subset_data[static_columns].values  # Shape: (num_clients, num_features)

# Create the time-dependent target matrix
time_data = subset_data[time_columns].values  # Shape: (num_clients, num_time_steps)

print("Static Data Shape:", static_data.shape)  # (num_clients, num_features)
print("Time Data Shape:", time_data.shape)  # (num_clients, num_time_steps)


Static Data Shape: (1000, 4)
Time Data Shape: (1000, 31)


In [ ]:
X, y = [], []
sequence_length = 24  # Length of each input sequence (e.g., one year)

# Iterate over clients
for client_idx in range(static_data.shape[0]):  # Iterate over each client
    client_targets = time_data[client_idx]  # Shape: (num_time_steps,)

    # Create sequences for this client
    for t in range(len(client_targets) - sequence_length):  # Slide over time steps
        # Static features repeated for each time step in the sequence
        static_features = static_data[client_idx]  # Shape: (num_features,)
        static_repeated = np.tile(static_features, (sequence_length, 1))  # Shape: (seq_len, num_features)
    
        # Ensure the static features are of type float32 (numeric)
        static_repeated = static_repeated.astype(np.float32)
    
        # Combine static and time-dependent features
        X.append(static_repeated)  # Static features remain constant for each sequence
        y.append(client_targets[t + sequence_length])  # Target is the value at the next time step
        # y.append(torch.ones(1).unsqueeze(0))

# Convert to numpy arrays
X = np.array(X)  # Shape: (num_samples, seq_len, num_features)
y = np.array(y)  # Shape: (num_samples,)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)  # Shape: (num_samples, seq_len, input_size)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Shape: (num_samples, 1)

print("Input Shape:", X.shape)  # (num_samples, seq_len, input_size)
print("Target Shape:", y.shape)  # (num_samples, 1)

# Hyperparameters
input_size = static_data.shape[1]  # Number of static features
hidden_size = 32  # Size of LSTM hidden states
num_layers = 5    # Number of LSTM layers
output_size = 1   # Predicting a single value (next step's target)
batch_size = 4    # Number of samples per batch
epochs = 10
learning_rate = 0.001

# Create DataLoader
dataset = torch.utils.data.TensorDataset(X, y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function, and optimizer
model = StackedLSTM(input_size, hidden_size, num_layers=5)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Initialize hidden and cell states
def init_hidden_states(batch_size, num_layers, hidden_size, device):
    return [(torch.zeros(batch_size, hidden_size).to(device), torch.zeros(batch_size, hidden_size).to(device)) 
            for _ in range(num_layers)]

for epoch in range(epochs):
    all_predictions = []
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(torch.float32), batch_y.to(torch.float32)

        # Initialize hidden states for the batch
        hidden_states = init_hidden_states(batch_size, num_layers=5, hidden_size=hidden_size, device=batch_X.device)

        # Forward pass
        outputs = model(batch_X, hidden_states)
        all_predictions.append(outputs.detach().cpu().numpy())
        loss = criterion(outputs, batch_y)  # Direct comparison with targets

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
    
    # After the loop, concatenate all predictions
    all_predictions = np.concatenate(all_predictions, axis=0)

    # Calculate statistics
    avg_prediction = np.mean(all_predictions)
    min_prediction = np.min(all_predictions)
    max_prediction = np.max(all_predictions)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
    print(f"Predictions - Average: {avg_prediction:.4f}, Min: {min_prediction:.4f}, Max: {max_prediction:.4f}")


Input Shape: torch.Size([7000, 24, 4])
Target Shape: torch.Size([7000, 1])


In [130]:
list(subset_data.columns)

['entite_eco',
 'personne_morale',
 'annee_soins',
 'colloc',
 'adh_fac',
 'type_cont',
 'code_postal',
 'Age22',
 'tranche_age22',
 'rg_benef',
 'PRES2201',
 'PRES2202',
 'PRES2203',
 'PRES2204',
 'PRES2205',
 'PRES2206',
 'PRES2207',
 'PRES2208',
 'PRES2209',
 'PRES2210',
 'PRES2211',
 'PRES2212',
 'nb_occ_id',
 'ra_0122',
 'ra_0222',
 'ra_0322',
 'ra_0422',
 'ra_0522',
 'ra_0622',
 'ra_0722',
 'ra_0822',
 'ra_0922',
 'ra_1022',
 'ra_1122',
 'ra_1222',
 'fra_0122',
 'fra_0222',
 'fra_0322',
 'fra_0422',
 'fra_0522',
 'fra_0622',
 'fra_0722',
 'fra_0822',
 'fra_0922',
 'fra_1022',
 'fra_1122',
 'fra_1222',
 'delta_0122',
 'delta_0222',
 'delta_0322',
 'delta_0422',
 'delta_0522',
 'delta_0622',
 'delta_0722',
 'delta_0822',
 'delta_0922',
 'delta_1022',
 'delta_1122',
 'delta_1222',
 'PRES2301',
 'PRES2302',
 'PRES2303',
 'PRES2304',
 'PRES2305',
 'PRES2306',
 'PRES2307',
 'PRES2308',
 'PRES2309',
 'PRES2310',
 'PRES2311',
 'PRES2312',
 'ra_0123',
 'ra_0223',
 'ra_0323',
 'ra_0423',
 